<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/hyper_Parameter_%EB%AC%B4_%EC%8B%9C%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
# 코랩 한글깨짐 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [ ]:
from google.colab import files
uploaded = files.upload()

무

In [ ]:
dataset= pd.read_excel('/content/무_시장.xlsx')
#배추= pd.read_excel('/content/배추_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.info()
# dataset.corr()

In [ ]:
# 타겟변수

In [ ]:
from scipy import stats
from scipy.stats import norm
sns.distplot(dataset['가격'],fit=norm, color="green")

In [ ]:
stats.probplot(dataset['가격'],plot=plt)

In [ ]:
dataset['가격'] = np.log1p(dataset['가격'])
sns.distplot(dataset['가격'],fit=norm,color="green")

In [ ]:
stats.probplot(dataset['가격'],plot=plt)

In [ ]:
sns.distplot(dataset['경락가평균가격'],fit=norm, color="green")

In [ ]:
stats.probplot(dataset['경락가평균가격'],plot=plt)

In [ ]:
dataset['경락가평균가격'] = np.log1p(dataset['경락가평균가격'])
sns.distplot(dataset['경락가평균가격'],fit=norm,color="green")

In [ ]:
stats.probplot(dataset['경락가평균가격'],plot=plt)

In [ ]:
sns.distplot(dataset['반입량'],fit=norm, color="green")

In [ ]:
stats.probplot(dataset['반입량'],plot=plt)

In [ ]:
dataset['반입량'] = np.log1p(dataset['반입량'])
sns.distplot(dataset['반입량'],fit=norm,color="green")

In [ ]:
stats.probplot(dataset['반입량'],plot=plt)

In [ ]:
sns.distplot(dataset['도매가격'],fit=norm, color="green")

In [ ]:
stats.probplot(dataset['도매가격'],plot=plt)

In [ ]:
dataset['도매가격']= np.log1p(dataset['도매가격'])
sns.distplot(dataset['도매가격'],fit=norm,color="green")

In [ ]:
stats.probplot(dataset['도매가격'],plot=plt)

In [ ]:
#Box-Cox변환은 y가 0이거나 0보다 작은 경우는 적용할 수 없다는 한계가 있다.

In [ ]:
datasetbox=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]

In [ ]:
from scipy.stats import skew 
numeric_features = datasetbox.dtypes[datasetbox.dtypes != "object"].index
skewed_feats = datasetbox[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness= pd.DataFrame({'Skew':skewed_feats})
skewness

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print(skewness)

In [ ]:
print("박스-콕스 변환이 필요한 수치형 변수의 개수는 {}으로 확인된다. ".format(skewness.shape[0]))

In [ ]:
from scipy.special import boxcox
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    datasetbox[feat]=boxcox(datasetbox[feat],lam)

In [ ]:
datasetbox.head()

In [ ]:
sns.distplot(datasetbox['가격'])
plt.show()

In [ ]:
datasetlog=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

In [ ]:
datasetlog.head()

In [ ]:
sns.distplot(datasetlog['일강수량(mm)'])

In [ ]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaler.fit(datasetlog)
datasetlog1 = scaler.transform(datasetlog)

datasetlog1 = pd.DataFrame(data=datasetlog1, columns=['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)'])
print('feature들의 최소 값')
print(datasetlog1.min())
print('\nfeature들의 최대 값')
print(datasetlog1.max())

In [ ]:
datasetlog1.head()

In [ ]:
dataset=pd.concat([datasetbox,datasetlog1], axis=1)

In [ ]:
dataset.head()

In [ ]:
f, axes = plt.subplots(3, 3, figsize=(15, 15), sharex=True)
sns.distplot(datasetbox["가격"], hist=True,kde=True,color="blue",ax=axes[0, 0])
sns.distplot(datasetbox["경락가평균가격"], hist=True,kde=True,color="green",ax=axes[0, 1])
sns.distplot(datasetbox["반입량"], hist=True,kde=True,color="yellow",ax=axes[0, 2])
sns.distplot(datasetbox["유가 전국평균가격"], hist=True,kde=True,color="green",ax=axes[1, 0])
sns.distplot(datasetbox["도매가격"], hist=True,kde=True,color="yellow",ax=axes[1, 1])
sns.distplot(datasetlog1["유무"], hist=True,kde=True,color="blue",ax=axes[1,2])
sns.distplot(datasetlog1["최저기온(°C)"], hist=True,kde=True,color="blue",ax=axes[2,0])
sns.distplot(datasetlog1["최고기온(°C)"], hist=True,kde=True,color="green",ax=axes[2, 1])
sns.distplot(datasetlog1["일강수량(mm)"], hist=True,kde=True,color="yellow",ax=axes[2, 2])
plt.show()

In [ ]:
f, axes = plt.subplots(3, 3, figsize=(20, 20), sharex=True)
sns.distplot(dataset["가격"], hist=True,kde=True,bins=10,color="blue",ax=axes[0, 0])
sns.distplot(dataset["경락가평균가격"], hist=True,kde=True,bins=10,color="green",ax=axes[0, 1])
sns.distplot(dataset["반입량"], hist=True,kde=True,bins=10,color="yellow",ax=axes[0, 2])
sns.distplot(dataset["유가 전국평균가격"], hist=True,kde=True,bins=10,color="green",ax=axes[1, 0])
sns.distplot(dataset["도매가격"], hist=True,kde=True,bins=10,color="yellow",ax=axes[1, 1])
sns.distplot(dataset["유무"], hist=True,kde=True,bins=10,color="blue",ax=axes[1,2])
sns.distplot(dataset["최저기온(°C)"], hist=True,kde=True,bins=10,color="blue",ax=axes[2,0])
sns.distplot(dataset["최고기온(°C)"], hist=True,kde=True,bins=10,color="yellow",ax=axes[2, 1])
sns.distplot(dataset["일강수량(mm)"], hist=True,kde=True,bins=10,color="yellow",ax=axes[2, 2])
#sns.distplot(dataset["유가 전국평균가격"], color="green", ax=axes[1, 0])
#sns.distplot(dataset["도매가격"], color="green", ax=axes[1, 1])
#sns.distplot(dataset["유무"], color="green", ax=axes[1, 2])
#sns.distplot(dataset["최저기온(°C)"], color="green", ax=axes[2, 0])
#sns.distplot(dataset["최고기온(°C)"], color="green", ax=axes[2, 1])
#sns.distplot(dataset["일강수량(mm)"], color="green", ax=axes[2, 2])
plt.show()

In [ ]:
sns.distplot(dataset)

In [ ]:
무_더미 = pd.read_excel('/content/무_더미데이터.xlsx')
무_더미.info()

In [ ]:
무_더미.drop(['일자'], axis=1, inplace=True)

In [ ]:
무_더미.head()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
X_data

In [ ]:
dataset

In [ ]:
# 변수변 왜도 확인

In [ ]:
#y_target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

## XGBoost Regressor 하이퍼파라미터


In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
#xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
#xgb_parameter

In [ ]:
#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

In [ ]:
#lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
#lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

#gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
#gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.7, 
                       gamma = 0.1,
                       learning_rate = 0.03,
                       max_depth = 3,
                       min_child_weight = 5,
                       n_estimators = 200,
                       objective = 'reg:squarederror',
                       subsample = 0.7)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 7,
                               max_features = 3,
                               min_samples_leaf = 3,
                               min_samples_split = 8,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.7,
                        gamma = 0.1,
                        learning_rate = 0.1,
                        max_depth = 7,
                        min_child_samples = 20,
                        n_estimators = 100,
                        num_leaves =8,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.1,
                                   max_depth=7,
                                   n_estimators=100,
                                   subsample=0.7
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
xgb_reg = XGBRegressor(colsample_bytree = 0.7, 
                       gamma = 0.1,
                       learning_rate = 0.03,
                       max_depth = 3,
                       min_child_weight = 5,
                       n_estimators = 200,
                       objective = 'reg:squarederror',
                       subsample = 0.7)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingRegressor

er = VotingRegressor([('xgb_reg', xgb_reg), ('gb_reg', gb_reg),('lgb_reg',lgb_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
##########예측

In [ ]:
ypred = xgb_reg.predict(무_더미)
ypred
#mse = mean_squared_error(y_test, ypred)

In [ ]:
무_더미['예측가격'] = ypred

In [ ]:
무_더미.to_excel('무_더미(예측).xlsx')

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plot_importance(xgb_reg)
plt.show()

In [ ]:
dataset.head()

In [ ]:
y = dataset['가격']
x = dataset.drop(['가격','유무','최고기온(°C)','일강수량(mm)'], axis=1, inplace=False)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=140)
print(x_train.shape, x_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(x_train, y_train) 

In [ ]:
y_predict = mlr.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_predict, alpha=0.4)
plt.xlabel("Actual Rent")
plt.ylabel("Predicted Rent")
plt.title("MULTIPLE LINEAR REGRESSION")


In [ ]:
print(mlr.coef_)

In [ ]:
print(mlr.score(x_train, y_train))

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
res = ols('y~x', data=dataset).fit()

In [ ]:
res.summary()

In [ ]:
x

In [ ]:
abc = []
for alg in er.named_estimators:
    clf = er.named_estimators[alg]
    a = clf.__class__.__name__
    b = [pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])]
    abc.append({a:b})

abc

In [ ]:
무.corr()

In [ ]:
배추.head()

In [ ]:
df = 무.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(7,7) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

In [ ]:
df = 배추.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(7,7) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
무특= pd.read_excel('/content/무_특.xlsx')
#배추특= pd.read_excel('/content/배추_특.xlsx')

In [ ]:
무특.head()

In [ ]:
배추특.head()

In [ ]:
무특.drop(['요일','요일(1)','Unnamed: 14'], axis=1, inplace=True)

In [ ]:
배추특.drop(['합계 일조시간(hr)','평균 5cm 지중온도(°C)'], axis=1, inplace=True)
배추특['일교차']=배추특['최고기온(°C)']-배추특['최저기온(°C)']

In [ ]:
df = 무특.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(10,10) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

In [ ]:
df = 배추특.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(10,10) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

In [ ]:
무특.set_index('일자',inplace=True)
무특.head()

In [ ]:
무특=무특[['평균가격','반입량']].resample('MS').mean()

In [ ]:
무특=pd.DataFrame(무특)

In [ ]:
무특.reset_index('일자',inplace=True)

In [ ]:
무특.head()

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('default')
plt.rcParams['figure.figsize'] = (7, 5)
plt.rcParams['font.size'] = 10

x = 무특['일자']
y1 =무특['반입량']
y2 =무특['평균가격']

fig, ax1 = plt.subplots()
ax1.set_xlabel('일자')
ax1.set_ylabel('반입량')
line1 = ax1.plot(x, y1, color='green', label='반입량')

ax2 = ax1.twinx()
ax2.set_ylabel('평균가격')
line2 = ax2.plot(x, y2, color='deeppink', label='평균가격')

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')
plt.show()